In [13]:
from IPython.display import display
import warnings

from itertools import zip_longest
from dataclasses import dataclass, field 
from typing import List, Dict
import pandas as pd
import pulp

# options
warnings.filterwarnings("ignore")
pd.options.display.precision = 0

@dataclass
class QualitySolver:
    
    """ Class for optimization of LPG mix blending costs and monthly planning.
    
    For inventories input it's possible to either enter each product separately 
    by using the respective argument or to enter the list of all product quantities
    using 'inventories' argument in the following order:
    
    [VNPZ Mix, VNPZ Butane, PNOS Mix, PNOS Butane, TCO Propane, TCO Butane, NNK Butane, \
    NNK Iso-Butane, TUY Butane, SHKP Butane, RM Butane]
    
    """
    
    # import vessels requirements
    vessels: List[str]
    qtty   : List[int]
    qual   : List[str]
    
    
    # input prices
    sonatrach_c3: int = 660
    sonatrach_c4: int = 640
    
    
    # import quality specs
    input_quality : pd.DataFrame = pd.read_csv('quality.csv', nrows = 11) # insert valid path to file
      
    spec_standards: Dict = field(default_factory= lambda: {'Turk summer'    :  {'Sulfur content': 50,
                                                                                'Propane min': 40
                                                                               },
                                                  
                                                           'Turk winter'    :  {'Sulfur content': 50,
                                                                                'Propane min': 60
                                                                               },
                            
                                                           'EN 589'         :  {'Sulfur content': 30,
                                                                                'Propane min': 30,
                                                                                'Butadiene max': 0.1
                                                                               },

                                                           'Vitol LPG mix'  :  {'Sulfur content': 30,
                                                                                'Propane min': 80,
                                                                                'Butadiene max': 0.1,
                                                                                'Pentanes max': 0.5
                                                                               },

                                                           'Propane 30 ppm' :  {'Sulfur content': 30,
                                                                                'Propane min': 95,
                                                                                'Butadiene max': 0.1
                                                                               },

                                                           'Propane 50 ppm' :  {'Sulfur content': 50,
                                                                                'Propane min': 95,
                                                                                'Butadiene max': 0.1
                                                                               },

                                                           'Butane 50 ppm'  :  {'Sulfur content': 30,
                                                                                'Propane max': 5
                                                                               },

                                                           'Butane off-spec':  {'Sulfur content': 200,
                                                                                'Propane max': 5
                                                                               },
                                                          }) 

   
    # available product quantities / 1st option
    inventories: List = None
    
    # available product quantities / 2nd option
    VNPZ_MIX   : List = field(default_factory=lambda: [5000]) #VNPZ_mix_qtty()
    VNPZ_C4    : List = field(default_factory=lambda: [5000]) #VNPZ_C4_qtty()
    PNOS_PBT   : List = field(default_factory=lambda: [5000]) #PNOS_mix_qtty()
    PNOS_C4    : List = field(default_factory=lambda: [5000]) #PNOS_C4_qtty()
    TCO_C3     : List = field(default_factory=lambda: [5000]) #TCO_C3_qtty()
    TCO_C4     : List = field(default_factory=lambda: [5000]) #TCO_C4_qtty()
    NNK_C4     : List = field(default_factory=lambda: [5000]) #NNK_C4_qtty()
    NNK_ISO    : List = field(default_factory=lambda: [5000]) #NNK_ISO_qtty()
    TUY_C4     : List = field(default_factory=lambda: [5000]) #TUY_C4_qtty()
    SHKP_C4    : List = field(default_factory=lambda: [5000]) #SHKP_C4_qtty()
    RM_C4      : List = field(default_factory=lambda: [5000]) #RM_C4_qtty()
    
    
    # additional costs (mean RTC demurrage amounts per mt)
    TCO_RTC    : int  = 5.33
    Lukoil_RTC : int  = 11.6
    Rosneft_RTC: int  = 7.77
    RM_RTC     : int  = 4.49
    
    
    
    def data_preparation(self):
        
        """ This method produces dictionaries in the format suitable for PuLP solver """
        
        # inventory quality specs
        self.products     = list(self.input_quality['Product'])
        self.methane      = dict(zip(self.products, self.input_quality['Methane']))
        self.ethane       = dict(zip(self.products, self.input_quality['Ethane']))
        self.ethylene     = dict(zip(self.products, self.input_quality['Ethylene']))
        self.propane      = dict(zip(self.products, self.input_quality['Propane']))
        self.propylene    = dict(zip(self.products, self.input_quality['Propylene']))
        self.iso_butane   = dict(zip(self.products, self.input_quality['Iso - Butane']))
        self.butane       = dict(zip(self.products, self.input_quality['n - Butane']))
        self.trans_butene = dict(zip(self.products, self.input_quality['Trans - Butene - 2']))
        self.butene       = dict(zip(self.products, self.input_quality['Butene - 1']))
        self.iso_butene   = dict(zip(self.products, self.input_quality['Iso - Butene']))
        self.cis_butene   = dict(zip(self.products, self.input_quality['Cis - Butene - 2']))
        self.neo_pentane  = dict(zip(self.products, self.input_quality['Neo - Pentane']))
        self.iso_pentane  = dict(zip(self.products, self.input_quality['Iso - Pentane']))
        self.pentane      = dict(zip(self.products, self.input_quality['n - Pentane']))
        self.butadiene    = dict(zip(self.products, self.input_quality['1,3 - Butadiene']))
        self.olefins      = dict(zip(self.products, self.input_quality['Olefines Content']))
        self.sulfur       = dict(zip(self.products, self.input_quality['Sulfur Content']))
        
        # pricing formulas
        VNPZ_MIX_price    = (self.sonatrach_c3 * 0.6 + self.sonatrach_c4 * 0.4 - 85) + self.Lukoil_RTC
        VNPZ_C4_price     = (self.sonatrach_c3 * 0.1 + self.sonatrach_c4 * 0.9 - 85) + self.Lukoil_RTC
        PNOS_MIX_price    = (self.sonatrach_c3 * 0.6 + self.sonatrach_c4 * 0.4 - 80) + self.Lukoil_RTC
        PNOS_C4_price     = (self.sonatrach_c3 * 0.1 + self.sonatrach_c4 * 0.9 - 80) + self.Lukoil_RTC
        TCO_C3_price      = (self.sonatrach_c3 - 60) + self.TCO_RTC
        TCO_C4_price      = (self.sonatrach_c4 - 60) + self.TCO_RTC
        NNK_C4_price      = (self.sonatrach_c4 - 75) + self.Rosneft_RTC
        NNK_ISO_price     = (self.sonatrach_c4 - 75) + self.Rosneft_RTC
        TUY_C4_price      = (self.sonatrach_c4 - 75) + self.Rosneft_RTC
        SHKPV_C4_price    = (self.sonatrach_c4 - 75) + self.Rosneft_RTC
        RM_C4_price       = (self.sonatrach_c4 - 85) + self.RM_RTC
        
        _prices           = [VNPZ_MIX_price, VNPZ_C4_price, PNOS_MIX_price, PNOS_C4_price, TCO_C3_price,
                             TCO_C4_price, NNK_C4_price, NNK_ISO_price, TUY_C4_price, SHKPV_C4_price, RM_C4_price]
        
        self.costs        = dict(zip(self.products, _prices)) 
        
        # cargo requirements
        self.qtty_req     = dict(zip(self.vessels, self.qtty))
        self.qual_req     = dict(zip(self.vessels, self.qual))

        
    def problem_declaration(self):
        
        """ This method is used to declare variables and instantiate the problem for PuLP solver """
        
        # variables declaration
        self.qtty_vars = pulp.LpVariable.dicts('Quantity', (self.products, self.qtty_req), lowBound = 0, cat='Integer')

        # instantiate PULP problem
        self.prob = pulp.LpProblem('Optimal Product Distribution', pulp.LpMinimize)

        # set up objective function
        self.prob += pulp.lpSum(self.costs[i] * self.qtty_vars[i][j] for i in self.products for j in self.vessels)

        # constraint declarations
        for vessel in self.vessels:
            
            self.prob += pulp.lpSum(sum(x[vessel] for x in self.qtty_vars.values())) == self.qtty_req[vessel]

            # getting quality standard from pre-defined options
            if self.qual_req[vessel] in self.spec_standards.keys():
                spec = self.spec_standards[self.qual_req[vessel]]
            else:
                error_arg = self.qual_req[vessel]
                raise KeyError(f'Corresponding quality specification {error_arg} not found')
            
                
            # quality requirements
            sulfur     = spec['Sulfur content']
            min_C3     = spec['Propane min']   if 'Propane min'   in spec.keys() else 0
            max_C3     = spec['Propane max']   if 'Propane max'   in spec.keys() else 100
            butadiene  = spec['Butadiene max'] if 'Butadiene max' in spec.keys() else 100
            pentanes   = spec['Pentanes max']  if 'Pentanes max'  in spec.keys() else 100
            
            # quality consraints
            # sulfur content cap
            self.prob += pulp.lpSum(self.sulfur[i] * self.qtty_vars[i][vessel] for i in self.products) <= sulfur * self.qtty_req[vessel] 
            
            
            # minimum C3 and lighter
            self.prob += pulp.lpSum([pulp.lpSum(self.methane[i] * self.qtty_vars[i][vessel] for i in self.products),                              
                                     pulp.lpSum(self.ethane[i] * self.qtty_vars[i][vessel] for i in self.products),
                                     pulp.lpSum(self.ethylene[i] * self.qtty_vars[i][vessel] for i in self.products),
                                     pulp.lpSum(self.propane[i] * self.qtty_vars[i][vessel] for i in self.products),
                                     pulp.lpSum(self.propylene[i] * self.qtty_vars[i][vessel] for i in self.products)]) >= min_C3 * self.qtty_req[vessel] 
            
            
            # maximum C3 and lighter
            self.prob += pulp.lpSum([pulp.lpSum(self.methane[i] * self.qtty_vars[i][vessel] for i in self.products),                              
                                     pulp.lpSum(self.ethane[i] * self.qtty_vars[i][vessel] for i in self.products),
                                     pulp.lpSum(self.ethylene[i] * self.qtty_vars[i][vessel] for i in self.products),
                                     pulp.lpSum(self.propane[i] * self.qtty_vars[i][vessel] for i in self.products),
                                     pulp.lpSum(self.propylene[i] * self.qtty_vars[i][vessel] for i in self.products)]) <= max_C3 * self.qtty_req[vessel]
            
            
            # maximum 1,3-Butadiene content
            if butadiene:
                self.prob += pulp.lpSum(self.butadiene[i] * self.qtty_vars[i][vessel] for i in self.products) <= butadiene * self.qtty_req[vessel] 
                
                
            # maximum C5 and heavier content
            if pentanes:
                self.prob += pulp.lpSum([pulp.lpSum(self.pentane[i] * self.qtty_vars[i][vessel] for i in self.products),
                                         pulp.lpSum(self.neo_pentane[i] * self.qtty_vars[i][vessel] for i in self.products),
                                         pulp.lpSum(self.iso_pentane[i] * self.qtty_vars[i][vessel] for i in self.products)]) <= pentanes * self.qtty_req[vessel] 

    
    def inventory_constraints(self):
        
        """ This method is used to declare inventory quantity constraints """
        
        # unpacking list of values to respective variables if "inventories" arg is supplied
        if self.inventories:
            self.VNPZ_MIX, self.VNPZ_C4, self.PNOS_PBT, self.PNOS_C4, self.TCO_C3, \
            self.TCO_C4, self.NNK_C4, self.NNK_ISO, self.TUY_C4, self.SHKP_C4, self.RM_C4 = self.inventories
            
        
        self.prob += sum(self.qtty_vars['VNPZ_Mix'].values())      <= self.VNPZ_MIX
        self.prob += sum(self.qtty_vars['VNPZ_C4'].values())       <= self.VNPZ_C4
        self.prob += sum(self.qtty_vars['PNOS_Mix'].values())      <= self.PNOS_PBT
        self.prob += sum(self.qtty_vars['PNOS_C4'].values())       <= self.PNOS_C4
        self.prob += sum(self.qtty_vars['TCO_C3'].values())        <= self.TCO_C3
        self.prob += sum(self.qtty_vars['TCO_C4'].values())        <= self.TCO_C4
        self.prob += sum(self.qtty_vars['Sanors_C4'].values())     <= self.NNK_C4
        self.prob += sum(self.qtty_vars['Sanors_Iso-C4'].values()) <= self.NNK_ISO
        self.prob += sum(self.qtty_vars['Tuymazy_C4'].values())    <= self.TUY_C4
        self.prob += sum(self.qtty_vars['Shkapovo_C4'].values())   <= self.SHKP_C4
        self.prob += sum(self.qtty_vars['Roza_Mira_C4'].values())  <= self.RM_C4

        
    def solve_problem(self):
        
        """ This method tries to find solution to the given model satisfying all constraints """
        
        self.prob.writeLP('optimal_loading.lp')
        #self.prob.solve(pulp.GLPK_CMD(path="C:\\Users\\bcherkasov\\Downloads\\winglpk\\glpk-4.55\\w64\\glpsol.exe", msg=1, timeLimit=60))
        self.prob.solve(pulp.PULP_CBC_CMD(timeLimit=60))
        
        if pulp.LpStatus[self.prob.status] == 'Optimal':
            self.solution_dataframe()
        else:
            print('Solution not found!')
            
    
    def solution_dataframe(self):
        
        """ This method creates a dataframe repr of solution """
        
        solution = []
        
        for var in self.prob.variables():
            solution.append(var.varValue)
        
        # as variables are sorted alphanumerically, 
        cols = [*sorted([vessel for vessel in self.vessels])]
        data = list(self.grouper(len(self.vessels), solution))
        index = sorted(self.products)
        
        self.df = pd.DataFrame(data=data, columns=cols, index=index)
        self.df['Total product qtty'] = self.df.sum(axis=1)
        
        inventories = [ self.PNOS_C4, self.PNOS_PBT, self.RM_C4, self.NNK_C4, self.NNK_ISO, self.SHKP_C4, 
                        self.TCO_C3, self.TCO_C4, self.TUY_C4, self.VNPZ_C4, self.VNPZ_MIX ]
        
        self.df['Rolling reserves'] = pd.Series(inventories).values - self.df['Total product qtty']
        self.df.loc['Totals'] = self.df.sum()
        display(self.df)
        
        total_cost = pulp.value(self.prob.objective)
        print(f'The cost of all input materials is {total_cost:,.2f} USD')
        
        
        
    def grouper(self, n, iterable, fillvalue=None):
        
        """ This is a helper method to format the solution more suitable for DataFrame format 
            grouper(3, 'ABCDEFG', 'x') --> ABC DEF Gxx """
        
        args = [iter(iterable)] * n
        return zip_longest(fillvalue=fillvalue, *args)
    
    
    def run(self):
        
        self.data_preparation()
        self.problem_declaration()
        self.inventory_constraints()
        self.solve_problem()
        


In [14]:
# Example of a typical monthly planning
vessels = ['09-01 Kuzguncuk', '09-03 Gaz Horizon', '09-06 Gremio', '09-10 Star Prime', '09-12 Kuzguncuk',
           '09-15 Eco Czar', '09-20 Gaz Horizon', '09-25 Kuzguncuk', '09-28 Star Prime']

qtty = [3000, 3700, 3500, 1700, 3100, 3500, 3700, 3000, 1700]

qual = ['Turk summer', 'Turk summer', 'Vitol LPG mix', 'Propane 50 ppm', 'Turk summer', \
        'EN 589', 'Turk summer', 'Propane 30 ppm', 'Butane off-spec']

inventories = [6000, 1000, 2000, 1500, 10000, 4000, 1500, 1500, 1500, 1500, 2000]

solver = QualitySolver(vessels=vessels, qtty=qtty, qual=qual, inventories=inventories)
solver.run()

,09-01 Kuzguncuk,09-03 Gaz Horizon,09-06 Gremio,09-10 Star Prime,09-12 Kuzguncuk,09-15 Eco Czar,09-20 Gaz Horizon,09-25 Kuzguncuk,09-28 Star Prime,Total product qtty,Rolling reserves
PNOS_C4,0,30,41,0,17,132,4,0,0,224,1276
PNOS_Mix,0,1017,0,0,0,983,0,0,0,2000,0
Roza_Mira_C4,1,102,0,2,0,1894,0,1,0,2000,0
Sanors_C4,1060,62,26,0,352,0,0,0,0,1500,0
Sanors_Iso-C4,730,0,7,32,3,0,447,63,218,1500,0
Shkapovo_C4,2,0,0,0,15,1,0,0,1482,1500,0
TCO_C3,1207,779,2842,1648,3,306,0,2891,0,9676,324
TCO_C4,0,0,0,0,0,0,0,0,0,0,4000
Tuymazy_C4,0,892,584,18,0,0,0,6,0,1500,0
VNPZ_C4,0,817,0,0,0,183,0,0,0,1000,0


The cost of all input materials is 15,748,659.48 USD


In [ ]:
import datetime as dt

print(dt.datetime.now())
solver.data_preparation()
print(dt.datetime.now())
solver.problem_declaration()
print(dt.datetime.now())
solver.inventory_constraints()
print(dt.datetime.now())
solver.solve_problem()
print(dt.datetime.now())

2022-08-16 15:48:46.763204
2022-08-16 15:48:46.763204
2022-08-16 15:48:46.777167
2022-08-16 15:48:46.778578


In [15]:
solver_list = pulp.listSolvers()
solver_list

['GLPK_CMD',
 'PYGLPK',
 'CPLEX_CMD',
 'CPLEX_PY',
 'GUROBI',
 'GUROBI_CMD',
 'MOSEK',
 'XPRESS',
 'PULP_CBC_CMD',
 'COIN_CMD',
 'COINMP_DLL',
 'CHOCO_CMD',
 'MIPCL_CMD',
 'SCIP_CMD']